<a href="https://colab.research.google.com/github/Jvlia17/blab/blob/main/Emotion_analysis_with_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, Embedding, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.initializers import Constant

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 2000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.2

Download the data

In [2]:
# Delete existing folders
!rm -rf sample_data

# Get the data and unzip it
!wget https://www.dropbox.com/s/845gac1uofd2mmz/NLP.zip?dl=0
!unzip /content/NLP.zip?dl=0

--2023-05-29 10:01:03--  https://www.dropbox.com/s/845gac1uofd2mmz/NLP.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/845gac1uofd2mmz/NLP.zip [following]
--2023-05-29 10:01:04--  https://www.dropbox.com/s/raw/845gac1uofd2mmz/NLP.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2f19af16ebf2cade6e27f1ca38.dl.dropboxusercontent.com/cd/0/inline/B88AN57DtGfttW7AIW2FID2eFBXS-7wjy-rFYG3-DWHMdido5szv5TCaUzsuskR2EAVjGZU3vHNO8B5HTaMer0Ui9Oz5M-sNnRvS5He3pk6LHQTJdioUuRVIITlP1BCl4d3bA0sL3d_I11y5ctIJBulKs_clsGloy6rYeW25KJLUFg/file# [following]
--2023-05-29 10:01:04--  https://uc2f19af16ebf2cade6e27f1ca38.dl.dropboxusercontent.com/cd/0/inline/B88AN57DtGfttW7AIW2FID2eFBXS-7wjy-rFYG3-DWHMdido5szv5TCaUzsuskR2EAVjGZU3vHNO8B5HTaMer0U

Reading Dataset 

In [3]:
#Reading input csv files
train = pd.read_csv("train.csv", on_bad_lines="skip")
test = pd.read_csv("test.csv", on_bad_lines="skip")
train.head(10)
train.nunique()

text            35546
fear                2
disgust             2
anger               2
joy                 2
surprise            2
trust               2
sadness             2
anticipation        2
dtype: int64

In [4]:
# Do not change
preprocess = False
#Assinging the texts to list of strings
train_texts = train.text.values
test_texts = test.text.values
#Assignings the labels as a separate df
labels = ['fear','disgust','anger','joy','surprise','trust','sadness','anticipation']
train_labels = train[labels]
test_labels = test[labels]
#Printing the list of strings
print("First comment text in training set:\n\n", test_texts[0])

First comment text in training set:

 @kole_holland don't fret  there's another game on at 8 #staytuned


Optional - Preprocessing

In [5]:

import re
import nltk
import string
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Does it make sense to make preprocessor a class, since it will mosty likely have only constant attributes, which could be just global variables?
# Probably not, although if at some point necessity for an internal variable for instance appears, class objects will allow it and global variables would not
class Preprocessor:
    def __init__(self):
        self.__token_url = "URL"
        self.__token_user = "USER"
    def __strip_urls(self, text):
        regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        urls = re.findall(regex,text)
        for url in urls:
            text = text.replace(url[0], self.__token_url)
        return text
    def __strip_users(self, text):
        for word in text.split():
            if(word[0] == "@"):
                text = text.replace(word, self.__token_user)
        return text
    def __strip_stopwords(self, text):
        stopwords = set(nltk.corpus.stopwords.words('english'))
        words = []
        for word in self.WordTokenize(text):
            if word not in stopwords:
                words.append(word)
        return ' '.join(words)
    def __lemmatize(self, text):
        words = []
        lemmatize = nltk.WordNetLemmatizer().lemmatize
        for word in self.WordTokenize(text):
            words.append( lemmatize(word) )
        return ' '.join(words)
    def __strip_punctuation(self, text):
        punctuation = string.punctuation
        return text.translate(str.maketrans('', '', punctuation))
    def WordTokenize(self, text):
        return word_tokenize(text)
    def Preprocess(self, text):
        text = text.lower()
        text = self.__strip_urls(text)
        text = self.__strip_users(text)
        text = self.__lemmatize(text)
        text = self.__strip_punctuation(text)
        text = self.__strip_stopwords(text)
        return text

# Preprocessing itself
p = Preprocessor()
preprocess = True
train_texts = [ p.Preprocess(i) for i in train_texts ]
test_texts = [ p.Preprocess(i) for i in test_texts ]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Vectorizers - use only one, depending which task you're solving

In [6]:
# For embedding (word2vec, glove, fasttext)
class EmbedVectorizer:
  def __init__(self):
    self.__v = Tokenizer(num_words = MAX_NUM_WORDS)
    self.DO_NOT_USE = self.__v
  def fit(self, texts):
    self.__v.fit_on_texts(texts)
  def convert(self, texts):
    return pad_sequences(self.__v.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)
vectorizer = EmbedVectorizer()

In [ ]:
# For Bag of Words
class BowVectorizer:
  def __init__(self):
    self.__v = Tokenizer(num_words = MAX_NUM_WORDS)
  def fit(self, texts):
    self.__v.fit_on_texts(texts)
  def convert(self, texts):
    return self.__v.texts_to_matrix(texts, mode = 'count')
vectorizer = BowVectorizer()

In [ ]:
# For TF-IDF
class TfIdfVectorizer:
  def __init__(self):
    self.__v = Tokenizer(num_words = MAX_NUM_WORDS)
  def fit(self, texts):
    self.__v.fit_on_texts(texts)
  def convert(self, texts):
    return self.__v.texts_to_matrix(texts, mode = 'tfidf')
vectorizer = TfIdfVectorizer()

Use vectorizer to convert strings into feature vectors

In [7]:
#Updates internal vocabulary based on a list of texts.
vectorizer.fit(train_texts)
#Transforms each text in texts to a sequence of integers.
train_sequences = vectorizer.convert(train_texts)
test_sequences = vectorizer.convert(test_texts)

In [8]:
#Display vectorized data (just for debugging) 
for i in train_sequences[0:5]:
  print(i)

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

**OSADZANIE SŁÓW: Defining a 1D CNN model for Embedding**

Model dla customowego (autorskiego) osadzania słów

In [ ]:
cnn_model = Sequential()
cnn_model.add(Embedding(MAX_NUM_WORDS, EMBEDDING_DIM))

Podejście Glove

In [9]:
# code to download glove data
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-05-29 10:02:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-29 10:02:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-29 10:02:28--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [10]:
EMBEDDING_DIM = 100 

embeddings_index = dict()

f = open('glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype = 'float32') #store all excepy val[0]
    embeddings_index[word] = coefs #populate embedding dict
    
f.close()

print(f'Found {len(embeddings_index)} word vectors') 

# prepare embedding_matrix
embedding_matrix = np.zeros((MAX_NUM_WORDS,EMBEDDING_DIM))

for word,index in vectorizer.DO_NOT_USE.word_index.items():
    
    if index> MAX_NUM_WORDS -1: #bounding by max feat
        break
    else:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

Found 400000 word vectors


Model dla Glove

In [11]:
cnn_model = Sequential()
cnn_model.add(Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, trainable = False, embeddings_initializer = Constant(embedding_matrix)))

Podejście FastText

In [ ]:
# code to download data
!wget https://www.dropbox.com/s/q419rkb3zomh1gx/wiki.simple.vec.zip?dl=0
!unzip -q /content/wiki.simple.vec.zip?dl=0

--2023-05-28 19:23:13--  https://www.dropbox.com/s/q419rkb3zomh1gx/wiki.simple.vec.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/q419rkb3zomh1gx/wiki.simple.vec.zip [following]
--2023-05-28 19:23:13--  https://www.dropbox.com/s/raw/q419rkb3zomh1gx/wiki.simple.vec.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc680a51866136912203c832ad34.dl.dropboxusercontent.com/cd/0/inline/B87Lx3e2reVqD4e3IBRbVoIW-0uwHIrVneRZNGZwn-NpL9eQOgBhemgfi9JmXBNCSMIiJUdhH4zP96hzeYOtSnx2FBtiaCoYDzlgx3e1vts2HG4dqs8afvY-L3XHJGq7C5qscQgcTCJPGR78zuQV65EWdSe_yX8BcKS4BqJ6onafog/file# [following]
--2023-05-28 19:23:13--  https://uc680a51866136912203c832ad34.dl.dropboxusercontent.com/cd/0/inline/B87Lx3e2reVqD4e3IBRbVoIW-0uwHIrVneRZNGZwn-NpL9eQOgBhem

In [ ]:
import os, re, csv, math, codecs
from tqdm import tqdm

#load embeddings
embeddings_index = {}
f = codecs.open('/content/wiki.simple.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

111052it [00:10, 10446.23it/s]

found 111052 word vectors


In [ ]:
EMBEDDING_DIM = 300

#embedding matrix
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NUM_WORDS, len(vectorizer.DO_NOT_USE.word_index))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in vectorizer.DO_NOT_USE.word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 132


Model dla FastText

In [ ]:
cnn_model = Sequential()
cnn_model.add(Embedding(nb_words, EMBEDDING_DIM,
          weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))

Podejście Word2Vec

In [ ]:
EMBEDDING_DIM = 100

from gensim.models import Word2Vec

vocab_size = len(vectorizer.DO_NOT_USE.word_index) + 1

word2vec_model = Word2Vec([w.split() for w in train_texts], vector_size = EMBEDDING_DIM, window=3, min_count=1, workers=16)

embedding_matrix = np.zeros((vocab_size, 100))
for word, i in vectorizer.DO_NOT_USE.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]


Model dla Word2Vec

In [ ]:
cnn_model = Sequential()
cnn_model.add(Embedding(vocab_size, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))

CZĘSTOŚĆ SŁÓW: CNN  model for BoW / similar

In [ ]:
cnn_model = Sequential()
cnn_model.add(Input(shape = (MAX_NUM_WORDS,1)))

In [ ]:
cnn_model = Sequential()
cnn_model.add(Input(shape = (MAX_NUM_WORDS,1)))
cnn_model.add(Dense(units = 128, activation = 'relu'))

Compile and fit the CNN model

In [12]:
cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = "relu"))
cnn_model.add(MaxPooling1D(pool_size = 5))
cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = "relu"))
cnn_model.add(MaxPooling1D(pool_size = 5))
cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = "relu"))
cnn_model.add(GlobalMaxPool1D())
cnn_model.add(Dense(units = 128, activation = 'relu'))
cnn_model.add(Dense(units = len(labels), activation = 'softmax'))

print(cnn_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         200000    
                                                                 
 conv1d (Conv1D)             (None, None, 128)         64128     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 128)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 128)        0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, None, 128)         8

K-fold cross validation

In [13]:
from sklearn.model_selection import KFold
FOLDS_NUM = 5

train_labels_numpy = train_labels.to_numpy()
acc_history = []
loss_history = []
kfold = KFold(n_splits=FOLDS_NUM, shuffle=True)

for train, test in kfold.split(train_sequences, train_labels_numpy):
  #Configures the model for training
  cnn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
  es = EarlyStopping(monitor = "accuracy", min_delta = 0.01, patience = 10, verbose = 1, mode = 'auto')
  mc = ModelCheckpoint(monitor = "accuracy", filepath = "./bestmodel.h5", verbose = 1, save_best_only = True, mode = 'auto')
  cd = [es, mc]
  #Trains the model for a fixed number of epochs (iterations on a dataset)
  #history = cnn_model.fit(train_sequences[train], train_labels_numpy[train], batch_size = 128, epochs = 100, validation_data = (train_sequences[test], train_labels_numpy[test]), callbacks = [cd])
  history = cnn_model.fit(train_sequences[train], train_labels_numpy[train], batch_size = 128, epochs = 100, callbacks = [cd])
  #Save results
  scores = cnn_model.evaluate(train_sequences[test], train_labels_numpy[test])
  loss_history.append(scores[0])
  acc_history.append(scores[1])

acc_average = sum(acc_history) / len(acc_history)
acc_stddev = ( sum([((x - acc_average) ** 2) for x in acc_history]) / len(acc_history) ) ** 0.5
loss_average = sum(loss_history) / len(loss_history)
loss_stddev = ( sum([((x - loss_average) ** 2) for x in loss_history]) / len(loss_history) ) ** 0.5
print(f"Accuracy: {acc_average} +- {acc_stddev}")
print(f"Loss: {loss_average} +- {loss_stddev}")

Epoch 1/100
223/223 [==============================] - ETA: 0s - loss: 0.3463 - accuracy: 0.2853
Epoch 1: accuracy improved from -inf to 0.28532, saving model to ./bestmodel.h5
223/223 [==============================] - 19s 26ms/step - loss: 0.3463 - accuracy: 0.2853
Epoch 2/100
222/223 [============================>.] - ETA: 0s - loss: 0.2710 - accuracy: 0.4869
Epoch 2: accuracy improved from 0.28532 to 0.48685, saving model to ./bestmodel.h5
223/223 [==============================] - 5s 24ms/step - loss: 0.2710 - accuracy: 0.4868
Epoch 3/100
222/223 [============================>.] - ETA: 0s - loss: 0.2369 - accuracy: 0.5687
Epoch 3: accuracy improved from 0.48685 to 0.56872, saving model to ./bestmodel.h5
223/223 [==============================] - 6s 25ms/step - loss: 0.2370 - accuracy: 0.5687
Epoch 4/100
222/223 [============================>.] - ETA: 0s - loss: 0.2103 - accuracy: 0.6254
Epoch 4: accuracy improved from 0.56872 to 0.62565, saving model to ./bestmodel.h5
223/223 [===

Final validation

In [14]:
x_test = test_sequences
y_test = test_labels
print("test loss, test acc:", cnn_model.evaluate(x_test, y_test, batch_size = 128))

71/71 [==============================] - 1s 12ms/step - loss: 0.8724 - accuracy: 0.5302
test loss, test acc: [0.8723818063735962, 0.5302304625511169]


Final validation (but better, faster, stronger)

In [15]:
from sklearn.metrics import classification_report

x_test = test_sequences
y_test = test_labels

y_pred = cnn_model.predict(x_test, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=-1)
y_pred_array = []
for i in y_pred_bool:
  y_pred_array.append(np.zeros(len(labels)))
  y_pred_array[-1][i] = 1

print(classification_report(y_test, y_pred_array, target_names = labels))

71/71 [==============================] - 1s 10ms/step
              precision    recall  f1-score   support

        fear       0.55      0.50      0.52      1486
     disgust       0.61      0.55      0.58       746
       anger       0.57      0.56      0.57      1410
         joy       0.58      0.62      0.60      1483
    surprise       0.43      0.51      0.47       892
       trust       0.39      0.35      0.37       745
     sadness       0.59      0.62      0.60      1475
anticipation       0.39      0.36      0.37       744

   micro avg       0.53      0.53      0.53      8981
   macro avg       0.51      0.51      0.51      8981
weighted avg       0.53      0.53      0.53      8981
 samples avg       0.53      0.53      0.53      8981



Predicting and Submitting for Test Data

In [16]:
test_sentences = [ 
    "The beauty of this channel is that, even though I might be well versed with all of its content concept-wise, I still enjoy very much watching these videos. I'm just delighted to see how you choose to present concepts, the graphical effort to show them clearly, in coordination with speech. Sometimes in incredibly revelaing ways. This is what teaching is truly about, and it is kinda sad how often teachers themselves mistake their job as 'giving concepts to students' (...and testing them) (or something along those lines). Your work is very inspirational, instructive, enjoyable. I'm so happy that I know you - and big props to each one collaborating to make these videos. I'm confident in saying that you're like friends for many of us, even though we don't know you in person!", 
    "I did my masters in statistics an I still find these types of videos invaluable to refresh my own intuition." , 
    "I have never enjoyed a probability course this much. these visualizations are really amazing", 
    "Sooo.... who is going to write a Chrome Extension to do the math for us when shopping on Amazon?",
    "How cold have I become? I didn't want to lose you by what I'd done caught in the gray",
    "Even in your deepest pain, even in your weakest hour, in your darkest night, you are lovely",
    "I used to be an adventurer like you, but then I took an arrow in the knee",
    "Can someone love me?"
    ]
if preprocess:
  test_sentences = [ p.Preprocess(i) for i in test_sentences ]
o_test = np.array( [i for i in test_sentences] )
o_test_vec = vectorizer.convert(o_test)
o_test_results = np.argmax(cnn_model.predict(o_test_vec), axis=-1)

for i in range(0, len(o_test)):
  text = o_test[i]
  emotion = list(train_labels.columns)[o_test_results[i]]
  print(f"{emotion}: {text}")

1/1 [==============================] - 0s 117ms/step
joy: beauty channel even though might well versed content conceptwise still enjoy much watching video delighted see choose present concept graphical effort show clearly coordination speech sometimes incredibly revelaing way teaching truly kinda sad often teacher mistake job giving concept student testing something along line work inspirational instructive enjoyable happy know big prop one collaborating make video confident saying like friend many u even though nt know person
fear: master statistic still find type video invaluable refresh intuition
joy: never enjoyed probability course much visualization really amazing
anticipation: sooo going write chrome extension math u shopping amazon
sadness: cold become nt want lose done caught gray
sadness: even deepest pain even weakest hour darkest night lovely
disgust: used adventurer like took arrow knee
sadness: someone love


REFERENCE - CLASSICAL BAG-OF-WORDS APPROACH

In [ ]:
import nltk

all_words = []
for text in train_texts:
  all_words.extend( text.lower().split() )
all_words = nltk.FreqDist(all_words)
word_features = list(all_words)[:MAX_NUM_WORDS]

def text_features(text): 
    text_words = set(text.split()) 
    features = {}
    for word in text_words:
        features['contains({})'.format(word)] = (word in text_words)
    return features

train_set = []
for i in range(len(train_texts)):
  label = 0
  for j in range( len(train_labels.columns) ):
    if train_labels.iloc[i,j] == 1:
      label = j
  train_set.append( (train_texts[i], list(train_labels.columns)[label]) )
print(train_set[0:10])

test_set = []
for i in range(len(test_texts)):
  label = 0
  for j in range( len(test_labels.columns) ):
    if test_labels.iloc[i,j] == 1:
      label = j
  test_set.append( (test_texts[i], list(test_labels.columns)[label]) )
print(test_set[0:10])

train_set = [ ( text_features(text), emotion ) for (text, emotion) in train_set ]
test_set = [ ( text_features(text), emotion ) for (text, emotion) in test_set ]

classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

[('USER USER USER anychance addressing communication sent yesterday still nt contact', 'fear'), ('life matter black life matter white couple handed water protestors asian man climbed stop light sign solidarity white woman cheered word encouragement their…', 'joy'), ('rip georgefloyd adding though guy wa police officer know law happened public knew witness camera knew would justifiable knew would go prison get', 'trust'), ('congrats nicole bb18 fix nose 😒💁🏻 bitter petty', 'anger'), ('alm alm alm alm alm alm alm black matter george floyd live mattered bad cop killed anyway gon na stop trial country way cop matter guilt get trial guilty trial first', 'trust'), ('USER — make feel vigorous fine kill said hestia display despair rather joyful —', 'sadness'), ('finally decided wa partially due fact feel post something important least interesting share', 'joy'), ('really study today chemistry playing madden way fun', 'anger'), ('george floyd story infuriated already hate cop furthers belief abs